In [7]:
import numpy as np
import random
import pandas as pd
from time import time
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

In [8]:
df=pd.read_csv(r'data.csv')
df_=df.drop(['Unnamed: 0' , 'index'] , axis=1)
df_.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,label
0,1.209513,0.588798,-0.443880,-0.343434,-0.388780,0.536716,-0.182281,-0.350877,-0.079242,0.216120,...,-0.067324,-0.168748,0.051440,0.095163,0.702551,0.665296,-0.006653,0.016676,-0.042396,59
1,1.202297,0.488674,-0.410836,-0.214348,0.052494,-0.643376,-0.383589,0.043562,-0.026972,0.579985,...,-0.081455,-0.283867,0.056391,0.049701,0.722340,0.651799,-0.001985,0.003494,-0.046455,59
2,1.202216,0.488681,-0.410753,-0.214394,0.052418,-0.643113,-0.383646,0.043509,-0.026983,0.579919,...,-0.081453,-0.286589,0.057193,0.048437,0.723262,0.650831,-0.000834,0.001222,-0.045558,59
3,0.639710,0.780882,-0.259236,0.044096,-0.340990,-0.344603,0.021090,-0.507578,0.003611,0.365988,...,0.937915,1.021114,0.320031,-0.199392,0.943167,0.374000,-0.484240,0.040488,-0.112420,962
4,1.114609,0.469510,0.704568,-0.322730,-0.219503,-0.202586,0.041881,-0.336737,-0.110972,0.291625,...,0.008656,-0.215116,0.057639,0.105707,0.703229,0.664095,0.002103,0.000461,-0.022415,1068


In [4]:
#create first population of coromosoms 100*125000

w,h=125000,250
corom=[[0 for x in range(w)] for y in range(h)]

for i in range(len(corom)):
    corom[i]=np.random.choice([0,1], size=125000 , p=[0.9 , 0.1])


In [9]:
#define cost and fitness functions

distr = []
arange = []
pas = 0
sta = -0.90
for i in range(2000): 
    lenn =len(df_.iloc[0][df_.iloc[0]<sta])
    distr.append((lenn / 1250000) - pas)
    pas = np.sum(distr)
    arange.append(round(sta,2))
    sta +=0.05
    if sta>1.5:
        break
        
def cost1(ch):
#     print("c1: ",np.sum(ch)/125000)
    return np.sum(ch)/1250000


def cost2(ch):
    df=df_.iloc[np.where(ch==1)]
    summy = []
    for j in df.columns:
        j = int(j)
        sc=0
        pas=0
        count=0
        summ=[]
        for i in arange:
            lenn =df[df[j]<i].shape[0]
            pers = (lenn / df.shape[0]  ) - pas
            pas += pers
            summ.append(np.abs(pers - distrToTal[j][count]))
            count+=1
        summy.append(np.sum(summ) * pca_ratio[j])
        
    return 1-np.sum(summy)

# def cost2(ch):
#     d=df_.iloc[np.where(ch==1)]
#     sc=0
#     count=0
#     pas=0
#     for i in arange:
#         lenn = d[d<i].count()
# #         lenn = (lenn / np.sum(ch)) - pas
#         lenn = (lenn / 1250000) - pas
#         pas += lenn
#         sc += np.abs(lenn - distr[count])
#         count+=1
# #     print("c2: ", 1-np.mean(sc))
#     return 1-np.mean(sc)

def calc_fitness(corom):
    
    c1=[]
    c2=[]
    
    for i in range(len(corom)):
        c1.append(cost1(corom[i]))
        c2.append((cost2(corom[i])))
#         c2.append((cost2(corom[i])-0.85)*1)
    
    print('\n')
    print("c1 mean",np.mean(c1))
    print("c2 mean",np.mean(c2))
    return ((1-np.array(c1))+(1-np.array(c2)))/2

In [6]:
for i in tqdm(range(500)):
    print("ittrate ", i)
    a=calc_fitness(corom)
    print("mean fitness: ",np.mean(a))
    print("best fitness: ",max(a))
    print("mean sample len: ", sum(corom).sum()/250)
    
    
#     #tournoment
#     new_corom=[]
#     for ii in range(35):
#         a_=a
#         best_parents=[]
#         random_selection=random.sample(range(0,100) , 5)
#         best_parents.append(np.where(a_==max(a_[random_selection]))[0][0])
#         random_selection=random.sample(range(0,100) , 5)
#         best_parents.append(np.where(a_==max(a_[random_selection]))[0][0])
#         new_corom.append(np.concatenate((corom[best_parents[0]][:62500],corom[best_parents[1]][62500:])))
#         new_corom.append(np.concatenate((corom[best_parents[0]][62500:],corom[best_parents[1]][:62500])))

#     for ii in range(7):
#         cc=random.sample(range(0,124999) , 500)
#         for jj in (cc):
#             if new_corom[ii][jj]==1:
#                 new_corom[ii][jj]=0
   
    
    #find best parents
    best_parents=[]
    for ii in range(14):
        ix=int(np.where(a==max(a))[0][0])
        best_parents.append(ix)
        a[ix]=-99
    
    print("best sample len: ", corom[best_parents[0]].sum())
    print("==================================================")
    
    #generate new childs
    new_corom=[]
    for ii in best_parents:
        for jj in best_parents:
            if ii < jj:
                new_corom.append(np.concatenate((corom[ii][:62500],corom[jj][62500:])))
                new_corom.append(np.concatenate((corom[ii][62500:],corom[jj][:62500])))
    
    #mutation
    for ii in range(int(len(new_corom)*0.35)):
        cc=random.sample(range(0,124999) , 50)
        for jj in (cc):
            if new_corom[ii][jj]==1:
                new_corom[ii][jj]=0
    
    #add best parents to new list of coromosoms
    for ii in best_parents:
        new_corom.append(corom[ii])
        
    #add 22 random coromosoms to new list of coromosoms    
    for ii in range(54):
        new_corom.append(np.random.choice([0,1], size=125000 , p=[0.95 , 0.05]))
    
    corom=[]
    corom=new_corom

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]

ittrate  0


c1 mean 0.0100066976
c2 mean 0.9903803541333334
mean fitness:  0.4998064741333334
best fitness:  0.4998104148148148
mean sample len:  12508.372
best sample len:  12251


  0%|▏                                                                              | 1/500 [01:36<13:26:20, 96.96s/it]

ittrate  1


c1 mean 0.0087894752
c2 mean 0.9915523854222222
mean fitness:  0.49982906968888885
best fitness:  0.49990274074074076
mean sample len:  10986.844
best sample len:  6022


  0%|▎                                                                              | 2/500 [03:04<12:46:09, 92.31s/it]

...

  2%|█▌                                                                             | 10/500 [11:01<9:00:22, 66.17s/it]

ittrate  10


c1 mean 0.004808089600000001
c2 mean 0.9953861297777779
mean fitness:  0.4999028903111111
best fitness:  0.49990395555555556
mean sample len:  6010.112
best sample len:  5938


  2%|█▋                                                                             | 11/500 [12:00<8:54:06, 65.53s/it]

ittrate  11


KeyboardInterrupt: 

In [7]:
# corom.to_csv(r"coroms.csv")

In [12]:
a=calc_fitness(corom)



c1 mean 0.0032107999999999998
c2 mean 0.9969239314962962


In [13]:
ix=int(np.where(a==max(a))[0][0])
ix

22

In [14]:
inxs=np.where(corom[ix]==1)[0]
inxs

array([    12,    102,    133, ..., 124890, 124925, 124947], dtype=int64)

In [15]:
df.iloc[inxs].to_csv(r"genetics_sample.csv")